<a href="https://colab.research.google.com/github/nicolay-r/THOR-ECAC/blob/master/SemEval_2024_Task_3_FlanT5_Finetuned_Model_Usage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Initialize

Loadin model and tokenizer

In [ ]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

# Setup model path.
model_path = "nicolay-r/flan-t5-emotion-cause-thor-base"
# Setup device.
device = "cuda:0"

model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
model.to(device)

Setup `ask` method for generating LLM responses.

In [15]:
def ask(prompt):
  inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
  inputs.to(device)
  output = model.generate(**inputs, max_length=320, temperature=1)
  return tokenizer.batch_decode(output, skip_special_tokens=True)[0]

Setup Chain-of-Thought

In [19]:
def emotion_extraction_chain(context, target):
  # Setup labels.
  labels_list = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
  # Setup Chain-of-Thought
  step1 = f"Given the conversation {context}, which text spans are possibly causes emotion on {target}?"
  span = ask(step1)
  step2 = f"{step1}. The mentioned text spans are about {span}. Based on the common sense, what " + f"is the implicit opinion towards the mentioned text spans that causes emotion on {target}, and why?"
  opinion = ask(step2)
  step3 = f"{step2}. The opinion towards the text spans that causes emotion on {target} is {opinion}. " + f"Based on such opinion, what is the emotion state of {target}?"
  emotion_state = ask(step3)
  step4 = f"{step3}. The emotion state is {emotion_state}. Based on these contexts, summarize and return the emotion cause only." + "Choose from: {}.".format(", ".join(labels_list))
  # Return the final response.
  return ask(step4)

## Usage

In [20]:
# setup history context (conv_turn_1)
conv_turn_1 = "John: ohh you made up!"
# setup utterance.
conv_turn_2 = "Jake: yaeh, I could not be mad at him for too long!"
context = conv_turn_1 + conv_turn_2
target = conv_turn_2
flant5_response = emotion_extraction_chain(context, target)

# output response
print(f"Emotion state of the speaker of `{target}` is: {flant5_response}")

Emotion state of the speaker of `Jake: yaeh, I could not be mad at him for too long!` is: anger
